# Zip code & Tract merging Notebook

Adapted from [Sarah Wang's notebook](https://github.com/jcweaver/broadband-capstone/blob/main/notebooks/census_broadband_merge.ipynb)

This notebook does a bit of data cleaning to pad all zipcode, county and tract IDs with 0s where needed.

Then this notebook calculates the weighted average of each Census statistic for each zipcode and appends these columns to the broadband dataset.

In [1]:
## imports 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import os

## Reading in data & some cleaning to pad with zeroes

In [2]:
# get broadband data
broadband_data = pd.read_csv("../data/merged_broadband.csv", index_col=0)
broadband_data.head()
# zipcode

,Zip,Population,WiredCount_2020,Fwcount_2020,AllProviderCount_2020,Wired25_3_2020,Wired100_3_2020,All25_3_2020,All100_3,TestCount,...,All25_3_2015,All100_3.1,Total_Enrolled_Households,ST,COUNTY NAME,COUNTY ID,BROADBAND USAGE,ERROR RANGE (MAE)(+/-),ERROR RANGE (95%)(+/-),MSD
0,29639,1742.0,3.0,0.0,8.0,3.0,3.0,5.0,3.0,163.0,...,3.0,3.0,21.0,SC,Abbeville,45001,0.948,0.034,0.110,0.002
1,29620,12934.0,6.0,0.0,11.0,5.0,3.0,7.0,3.0,2536.0,...,3.0,3.0,542.0,SC,Abbeville,45001,0.398,0.002,0.007,0.000
2,29659,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,SC,Abbeville,45001,0.206,0.152,0.608,0.043
3,29638,2944.0,6.0,1.0,13.0,4.0,4.0,6.0,4.0,272.0,...,2.0,2.0,68.0,SC,Abbeville,45001,0.369,0.010,0.031,-0.001
4,29628,2759.0,4.0,0.0,8.0,3.0,2.0,5.0,2.0,100.0,...,3.0,3.0,102.0,SC,Abbeville,45001,0.221,0.014,0.043,0.000


In [3]:
broadband_data.Zip.value_counts()[broadband_data.Zip.value_counts() > 1]

56160    2
99644    2
55003    2
99563    2
56446    2
        ..
56347    2
56528    2
55071    2
56389    2
99550    2
Name: Zip, Length: 82, dtype: int64

In [4]:
broadband_data[broadband_data.Zip=="56318"][['ST', 'COUNTY NAME', 'COUNTY ID',
       'BROADBAND USAGE', 'ERROR RANGE (MAE)(+/-)', 'ERROR RANGE (95%)(+/-)',
       'MSD', ]]

,ST,COUNTY NAME,COUNTY ID,BROADBAND USAGE,ERROR RANGE (MAE)(+/-),ERROR RANGE (95%)(+/-),MSD


In [5]:
broadband_data[broadband_data.Zip=="99550"][['ST', 'COUNTY NAME', 'COUNTY ID',
       'BROADBAND USAGE', 'ERROR RANGE (MAE)(+/-)', 'ERROR RANGE (95%)(+/-)',
       'MSD', ]]

,ST,COUNTY NAME,COUNTY ID,BROADBAND USAGE,ERROR RANGE (MAE)(+/-),ERROR RANGE (95%)(+/-),MSD


In [6]:
## Dropping the County ID column based on some EDA is was sometimes wrong
broadband_data = broadband_data.drop(columns=["COUNTY ID"])
broadband_data = broadband_data.drop_duplicates()

In [7]:
## Notice how ME states only 4 digits but zipcodes should be 5 digits
broadband_data[broadband_data.ST=="ME"]

,Zip,Population,WiredCount_2020,Fwcount_2020,AllProviderCount_2020,Wired25_3_2020,Wired100_3_2020,All25_3_2020,All100_3,TestCount,...,Wired100_3_2015,All25_3_2015,All100_3.1,Total_Enrolled_Households,ST,COUNTY NAME,BROADBAND USAGE,ERROR RANGE (MAE)(+/-),ERROR RANGE (95%)(+/-),MSD
719,4236,4350.0,2.0,2.0,10.0,2.0,1.0,6.0,2.0,1414.0,...,0.0,1.0,0.0,50.0,ME,Androscoggin,0.236,0.010,0.031,-0.001
720,4258,2617.0,2.0,1.0,9.0,1.0,1.0,4.0,1.0,24.0,...,0.0,1.0,0.0,26.0,ME,Androscoggin,0.170,0.014,0.043,0.000
721,4250,4228.0,3.0,0.0,9.0,2.0,2.0,4.0,2.0,1962.0,...,0.0,1.0,0.0,140.0,ME,Androscoggin,0.130,0.010,0.031,-0.001
722,4282,5734.0,2.0,0.0,8.0,2.0,1.0,4.0,1.0,2104.0,...,0.0,1.0,0.0,66.0,ME,Androscoggin,0.161,0.006,0.018,-0.001
723,4210,23045.0,5.0,2.0,13.0,3.0,3.0,7.0,4.0,25452.0,...,0.0,1.0,0.0,956.0,ME,Androscoggin,0.756,0.002,0.005,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32594,4049,3659.0,3.0,0.0,9.0,2.0,1.0,4.0,1.0,734.0,...,0.0,1.0,0.0,46.0,ME,York,0.149,0.010,0.031,-0.001
32595,4046,7496.0,4.0,0.0,10.0,4.0,3.0,6.0,3.0,3900.0,...,0.0,1.0,0.0,49.0,ME,York,0.647,0.004,0.013,0.000
32596,4048,2946.0,3.0,0.0,9.0,2.0,1.0,4.0,1.0,916.0,...,0.0,1.0,0.0,47.0,ME,York,0.343,0.010,0.031,-0.001
32597,3907,894.0,3.0,0.0,8.0,3.0,2.0,5.0,2.0,176.0,...,0.0,1.0,0.0,2.0,ME,York,0.536,0.020,0.061,0.000


In [8]:
## Helper function
def add_front_padding(x):
    zipcode = str(x)
    while len(zipcode) < 5:
        zipcode = "0"+zipcode
    return zipcode

In [9]:
## Add front padding for US zip codes which all should be 5 digits
broadband_data["Zip"] = broadband_data["Zip"].apply(add_front_padding)
zipcode = broadband_data.Zip.unique()

In [10]:
## Confirm ME states start with 0
broadband_data[broadband_data.ST=="ME"]

,Zip,Population,WiredCount_2020,Fwcount_2020,AllProviderCount_2020,Wired25_3_2020,Wired100_3_2020,All25_3_2020,All100_3,TestCount,...,Wired100_3_2015,All25_3_2015,All100_3.1,Total_Enrolled_Households,ST,COUNTY NAME,BROADBAND USAGE,ERROR RANGE (MAE)(+/-),ERROR RANGE (95%)(+/-),MSD
719,04236,4350.0,2.0,2.0,10.0,2.0,1.0,6.0,2.0,1414.0,...,0.0,1.0,0.0,50.0,ME,Androscoggin,0.236,0.010,0.031,-0.001
720,04258,2617.0,2.0,1.0,9.0,1.0,1.0,4.0,1.0,24.0,...,0.0,1.0,0.0,26.0,ME,Androscoggin,0.170,0.014,0.043,0.000
721,04250,4228.0,3.0,0.0,9.0,2.0,2.0,4.0,2.0,1962.0,...,0.0,1.0,0.0,140.0,ME,Androscoggin,0.130,0.010,0.031,-0.001
722,04282,5734.0,2.0,0.0,8.0,2.0,1.0,4.0,1.0,2104.0,...,0.0,1.0,0.0,66.0,ME,Androscoggin,0.161,0.006,0.018,-0.001
723,04210,23045.0,5.0,2.0,13.0,3.0,3.0,7.0,4.0,25452.0,...,0.0,1.0,0.0,956.0,ME,Androscoggin,0.756,0.002,0.005,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32594,04049,3659.0,3.0,0.0,9.0,2.0,1.0,4.0,1.0,734.0,...,0.0,1.0,0.0,46.0,ME,York,0.149,0.010,0.031,-0.001
32595,04046,7496.0,4.0,0.0,10.0,4.0,3.0,6.0,3.0,3900.0,...,0.0,1.0,0.0,49.0,ME,York,0.647,0.004,0.013,0.000
32596,04048,2946.0,3.0,0.0,9.0,2.0,1.0,4.0,1.0,916.0,...,0.0,1.0,0.0,47.0,ME,York,0.343,0.010,0.031,-0.001
32597,03907,894.0,3.0,0.0,8.0,3.0,2.0,5.0,2.0,176.0,...,0.0,1.0,0.0,2.0,ME,York,0.536,0.020,0.061,0.000


In [11]:
print(f'Total number of unique zipcode in broadband data: {len(zipcode)}')

Total number of unique zipcode in broadband data: 32653


In [12]:
#get census data
census_data = pd.read_csv("../data/relabeled_census.csv")
census_data.head()


,NAME,total_pop2,median_age_overall,median_age_male,median_age_female,state,county,tract,employment_rate,median_income,...,pct_internet_broadband_satellite,pct_internet_only_satellite,pct_internet_other,pct_internet_no_subscrp,pct_internet_none,pct_computer,pct_computer_with_dialup,pct_computer_with_broadband,pct_computer_no_internet,pct_no_computer
0,"Census Tract 11, Jefferson County, Alabama",4781,39.0,42.5,38.1,1,73,1100,51.0,37030.0,...,9.022150,0.918422,0.0,1.134522,24.851432,80.821178,0.0,74.014046,6.807131,19.178822
1,"Census Tract 14, Jefferson County, Alabama",1946,44.3,40.5,49.1,1,73,1400,45.4,36066.0,...,4.901961,0.000000,0.0,2.083333,25.490196,85.661765,0.0,71.078431,14.583333,14.338235
2,"Census Tract 20, Jefferson County, Alabama",4080,34.0,31.0,36.4,1,73,2000,47.7,27159.0,...,4.651163,0.000000,0.0,0.000000,45.454545,71.317829,0.0,54.545455,16.772375,28.682171
3,"Census Tract 38.02, Jefferson County, Alabama",5291,35.8,31.7,37.3,1,73,3802,51.7,38721.0,...,3.959873,0.000000,0.0,6.335797,33.632524,85.744456,0.0,59.450898,26.293559,14.255544
4,"Census Tract 40, Jefferson County, Alabama",2533,52.1,51.6,53.8,1,73,4000,36.9,18525.0,...,4.548635,1.959412,0.0,5.108467,47.515745,63.051085,0.0,44.786564,18.264521,36.948915


In [13]:
print(f'Total number of rows in census data: {len(census_data.index)}')

Total number of rows in census data: 73056


In [14]:
## This dataset came from: https://mcdc.missouri.edu/applications/geocorr2018.html
#get zipcode, county name, and state for mapping
tract_data = pd.read_csv('../data/zip_conversation_data/all_states_zip_conversion.csv', converters={'zcta5' : lambda x: str(x)}, skiprows = [1])
tract_data = tract_data.rename(columns={"zcta5" : "Zip"})
tract_data


,Zip,county,tract,cntyname,zipname,intptlon,intptlat,pop10,afact,AFACT2
0,01001,25013,8132.05,Hampden MA,"Agawam Town, MA",-72.630818,42.051901,3775,0.225118,0.504611
1,01001,25013,8132.06,Hampden MA,"Agawam Town, MA",-72.638052,42.066801,297,0.017711,0.073478
2,01001,25013,8132.07,Hampden MA,"Agawam Town, MA",-72.636153,42.087216,4133,0.246467,0.808648
3,01001,25013,8132.08,Hampden MA,"Agawam Town, MA",-72.609240,42.056361,2918,0.174011,1.000000
4,01001,25013,8132.09,Hampden MA,"Agawam Town, MA",-72.612651,42.075544,5646,0.336693,1.000000
...,...,...,...,...,...,...,...,...,...,...
144119,99999,49045,1307.01,Tooele UT,99999,-112.225848,40.722768,18,0.003210,0.005396
144120,99999,49045,1307.03,Tooele UT,99999,-112.293643,40.034868,6,0.001070,0.000858
144121,99999,51001,9802.00,Accomack VA,99999,-75.511891,37.838676,5,0.000892,1.000000
144122,99999,56023,9782.00,Lincoln WY,99999,-110.665086,42.055432,6,0.001070,0.003009


In [15]:
print(f'Total number of unique zipcodes in mapping table: {len(tract_data.Zip.unique())}')


Total number of unique zipcodes in mapping table: 32846


In [16]:
## Census tracts are 6 digits long
## See: https://www.census.gov/programs-surveys/geography/guidance/geo-identifiers.html
## https://www2.census.gov/geo/pdfs/education/CensusTracts.pdf

def pad_tract(x):
    ## Use string formatting to ensure two decimal points at the end
    ## This will add 00s for appropriate tracts
    tract = "{:.2f}".format(x)
    
    ## Remove period
    tract = tract.replace(".", "")
    
    ##Add additional front padding if needed
    while len(tract) < 6:
        tract = "0" + tract
        
    return tract

In [17]:
pad_tract(18.00)

'001800'

In [18]:
## Before - See decimals & shorter length of some codes
tract_data["tract"]

0         8132.05
1         8132.06
2         8132.07
3         8132.08
4         8132.09
           ...   
144119    1307.01
144120    1307.03
144121    9802.00
144122    9782.00
144123      18.00
Name: tract, Length: 144124, dtype: float64

In [19]:
## After - see all 6 digits and no decimals
tract_data["tract"] = tract_data["tract"].apply(pad_tract)
tract_data["tract"]

0         813205
1         813206
2         813207
3         813208
4         813209
           ...  
144119    130701
144120    130703
144121    980200
144122    978200
144123    001800
Name: tract, Length: 144124, dtype: object

In [20]:
def pad_to_six(x):
    tract = str(x)
    while len(tract) < 6:
        tract = "0" + tract
    return tract

In [21]:
## Before - Notice 00s already at end for some but some are not yet 6 digits
census_data["tract"]

0          1100
1          1400
2          2000
3          3802
4          4000
          ...  
73051      1902
73052    980801
73053      1602
73054      1603
73055    965100
Name: tract, Length: 73056, dtype: int64

In [22]:
## The Census data already removed the decimals and added 0s at the end where needed for tracts but
## the tracts need to be prepended with 0s to get to 6 digits
census_data["tract"] = census_data["tract"].apply(pad_to_six)

## Updating Census Tracts Dataset with Geographic Changes in 2011 & 2012 

Note the geocorr mapping tool only provides 2010 tracts mapped to zipcodes. There are changes to a few tracts after 2010, which affects our 2019 data.

For more info, refer to:
* 2011 Changes: https://www.census.gov/programs-surveys/acs/technical-documentation/table-and-geography-changes/2011/geography-changes.html
* 2012 Changes: https://www.census.gov/programs-surveys/acs/technical-documentation/table-and-geography-changes/2012/geography-changes.html

We have reviewed all other changes after 2012-2019 and did not find any relevant tract changes.

### First, let's make sure the county codes look correct

We need the county codes to identify the correct tract codes to replace. This is because a tract 9401.00 may be in multiple counties.

In [23]:
## Notice how some county codes only have 4-digits, these should have a 0 pre-pended before merging
tract_data[tract_data.county<20000]

,Zip,county,tract,cntyname,zipname,intptlon,intptlat,pop10,afact,AFACT2
4186,06001,9003,460302,Hartford CT,"Avon, CT",-72.879362,41.763873,355,0.019309,0.088024
4187,06001,9003,462101,Hartford CT,"Avon, CT",-72.878749,41.786317,5833,0.317269,1.000000
4188,06001,9003,462102,Hartford CT,"Avon, CT",-72.906296,41.789819,4239,0.230568,1.000000
4189,06001,9003,462201,Hartford CT,"Avon, CT",-72.850445,41.798346,5256,0.285885,1.000000
4190,06001,9003,462202,Hartford CT,"Avon, CT",-72.815894,41.794739,2692,0.146424,1.000000
...,...,...,...,...,...,...,...,...,...,...
144019,99999,16079,960200,Shoshone ID,99999,-116.155171,47.886140,2,0.000357,0.000479
144020,99999,16083,001500,Twin Falls ID,99999,-114.730683,42.212968,2,0.000357,0.000749
144021,99999,16085,970100,Valley ID,99999,-116.045233,44.417043,1,0.000178,0.000381
144022,99999,17017,960300,Cass IL,99999,-90.346830,40.083678,2,0.000357,0.000607


In [24]:
def pad_county_code(x):
    county = str(x)
    while len(county) < 5:
        county = "0" + county
    return county

tract_data["county"] = tract_data.county.apply(pad_county_code)

## Confirm zeroes appropriately pre-pended
tract_data[tract_data["county"] == "09003"]

,Zip,county,tract,cntyname,zipname,intptlon,intptlat,pop10,afact,AFACT2
4186,06001,09003,460302,Hartford CT,"Avon, CT",-72.879362,41.763873,355,0.019309,0.088024
4187,06001,09003,462101,Hartford CT,"Avon, CT",-72.878749,41.786317,5833,0.317269,1.000000
4188,06001,09003,462102,Hartford CT,"Avon, CT",-72.906296,41.789819,4239,0.230568,1.000000
4189,06001,09003,462201,Hartford CT,"Avon, CT",-72.850445,41.798346,5256,0.285885,1.000000
4190,06001,09003,462202,Hartford CT,"Avon, CT",-72.815894,41.794739,2692,0.146424,1.000000
...,...,...,...,...,...,...,...,...,...,...
4862,06489,09003,430301,Hartford CT,"Southington, CT",-72.854174,41.566638,3293,0.102691,0.914976
4863,06489,09003,430302,Hartford CT,"Southington, CT",-72.872427,41.580643,1292,0.040291,0.468965
4864,06489,09003,430500,Hartford CT,"Southington, CT",-72.913091,41.601834,2580,0.080456,0.394073
4865,06489,09003,430601,Hartford CT,"Southington, CT",-72.890297,41.621677,5202,0.162223,1.000000


In [25]:
## Madison County, NY County code is: 36053
madison_pairs = {
    "940101" : "030101",
    "940102" : "030102",
    "940103" : "030103",
    "940200" : "030200",
    "940300" : "030300",
    "940401" : "030401",
    "940403" : "030403",
    "940600" : "030600",
    "940700" : "030402"
}

for old, new in madison_pairs.items():
    tract_data.loc[(tract_data.county=="36053") & (tract_data.tract==old), "tract"] = new
    
#tract_data.loc[(tract_data.county=="36053") & (tract_data.tract=="030402")] #= "030402"

In [26]:
## Oneida NY County code is: 36065

oneida_pairs = {
    "940000" : "024800",
    "940100" : "024700",
    "940200" : "024900"
}

for old, new in oneida_pairs.items():
    tract_data.loc[(tract_data.county=="36065") & (tract_data.tract==old), "tract"] = new

In [27]:
## Pima County, AZ 04019

pima_pairs = {
    "002701" : "002704",
    "002903" : "002906",
    "410501" : "004118",
    "410502" : "004121",
    "410503" : "004125",
    "470400" : "005200",
    "470500" : "005300"
}

for old, new in pima_pairs.items():
    tract_data.loc[(tract_data.county=="04019") & (tract_data.tract==old), "tract"] = new

In [28]:
## Los Angeles County Code = 06037

tract_data.loc[(tract_data.county=="06037") & (tract_data.tract=="930401"), "tract"] = "137000"

## Missing ZipCode EDA

In [29]:
## These zipcodes are in the broadband data but not in the mapping data
zips_not_in_mapping_data = broadband_data[~broadband_data.Zip.isin(tract_data.Zip)]
zips_not_in_mapping_data[["Zip", "ST", "COUNTY NAME", "Population"]]

## All of these have population of 0, except for Carson City which is NaN
## But this site: https://www.unitedstateszipcodes.org/89702/ says that Carson City zipcode has 0 population
## Let's drop these

,Zip,ST,COUNTY NAME,Population
903,22214,VA,Arlington,0.0
1727,76508,TX,Bell,0.0
4057,89702,NV,Carson City,NaN
5575,73019,OK,Cleveland,0.0
7101,53792,WI,Dane,0.0
13764,80419,CO,Jefferson,0.0
14815,98174,WA,King,0.0
14828,98195,WA,King,0.0
21006,10110,NY,New York,0.0
25586,84144,UT,Salt Lake,0.0


In [30]:
## These zipcodes are in the mapping data but not in the broadband data
zips_not_in_broadband_data = tract_data[~tract_data.Zip.isin(broadband_data.Zip)]
zips_not_in_broadband_data[["Zip", "zipname", "pop10", "afact", "AFACT2"]]


,Zip,zipname,pop10,afact,AFACT2
102,01066,"North Hatfield, MA (PO Boxes)",64,1.000000,0.019518
1806,02584,"Nantucket, MA",10,1.000000,0.008591
2777,03754,"Guild, NH (PO Boxes)",86,1.000000,0.013217
3177,04271,"Paris, ME (PO Boxes)",67,1.000000,0.012927
3377,04570,"Squirrel Island, ME",2,1.000000,0.000722
...,...,...,...,...,...
144119,99999,99999,18,0.003210,0.005396
144120,99999,99999,6,0.001070,0.000858
144121,99999,99999,5,0.000892,1.000000
144122,99999,99999,6,0.001070,0.003009


## Joining the Data

We will join the data using a left join to take all keys from the tract set and leave out the few keys we saw above that are in the broadband set but not the tract set.

In [31]:
#merge tract data with broadband data to map a tract with every Zip
#merged_zips_with_tracts = tract_data.merge(broadband_data, how = 'left', on="Zip")
#merged_zips_with_tracts

In [32]:
## Now I want to merge the Census data with this above dataset
## I'll need to merge on tract, and county code combination

## Notice how the county codes for the tract mapping dataset were 5 digits - This is a combination of a 2-digit
## state code pre-pended to a 3 digit county code. I need to modify the census data to match this format.
## I'll use the helper functions below


def pad_state(x):
    state = str(x)
    while len(state) < 2:
        state = "0" + state
    return state

def pad_county(x):
    county = str(x)
    while len(county) < 3:
        county = "0" + county
    return county


In [33]:
census_data[["county", "state", "tract"]]

,county,state,tract
0,73,1,001100
1,73,1,001400
2,73,1,002000
3,73,1,003802
4,73,1,004000
...,...,...,...
73051,21,56,001902
73052,21,56,980801
73053,25,56,001602
73054,25,56,001603


In [34]:
## Modifying the census dataset county code to be 5 digits

census_data["county"] = census_data["county"].apply(pad_county)
census_data["state"] = census_data["state"].apply(pad_state)

census_data[["county", "state", "tract"]]

,county,state,tract
0,073,01,001100
1,073,01,001400
2,073,01,002000
3,073,01,003802
4,073,01,004000
...,...,...,...
73051,021,56,001902
73052,021,56,980801
73053,025,56,001602
73054,025,56,001603


In [35]:
census_data["county"] = census_data["state"]+census_data["county"]
census_data[["county", "state", "tract"]]

,county,state,tract
0,01073,01,001100
1,01073,01,001400
2,01073,01,002000
3,01073,01,003802
4,01073,01,004000
...,...,...,...
73051,56021,56,001902
73052,56021,56,980801
73053,56025,56,001602
73054,56025,56,001603


In [36]:
## Now we can merge on tract & county codes

merged = tract_data.merge(census_data, how="left", on=["tract", "county"])
merged

,Zip,county,tract,cntyname,zipname,intptlon,intptlat,pop10,afact,AFACT2,...,pct_internet_broadband_satellite,pct_internet_only_satellite,pct_internet_other,pct_internet_no_subscrp,pct_internet_none,pct_computer,pct_computer_with_dialup,pct_computer_with_broadband,pct_computer_no_internet,pct_no_computer
0,01001,25013,813205,Hampden MA,"Agawam Town, MA",-72.630818,42.051901,3775,0.225118,0.504611,...,2.473615,0.494723,0.000000,3.100264,9.135884,92.183377,0.000000,87.763852,4.419525,7.816623
1,01001,25013,813206,Hampden MA,"Agawam Town, MA",-72.638052,42.066801,297,0.017711,0.073478,...,0.692695,0.000000,0.000000,0.000000,15.050378,91.687657,1.070529,82.745592,7.871537,8.312343
2,01001,25013,813207,Hampden MA,"Agawam Town, MA",-72.636153,42.087216,4133,0.246467,0.808648,...,3.016453,0.000000,0.000000,7.358318,11.700183,89.762340,0.639854,80.301645,8.820841,10.237660
3,01001,25013,813208,Hampden MA,"Agawam Town, MA",-72.609240,42.056361,2918,0.174011,1.000000,...,0.577558,0.000000,0.000000,0.000000,8.415842,91.914191,0.000000,89.191419,2.722772,8.085809
4,01001,25013,813209,Hampden MA,"Agawam Town, MA",-72.612651,42.075544,5646,0.336693,1.000000,...,3.158677,0.000000,0.000000,5.388331,18.543292,82.683017,0.594575,74.284653,7.803790,17.316983
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144119,99999,49045,130701,Tooele UT,99999,-112.225848,40.722768,18,0.003210,0.005396,...,11.130742,0.000000,0.000000,0.530035,11.219081,94.346290,0.000000,88.250883,6.095406,5.653710
144120,99999,49045,130703,Tooele UT,99999,-112.293643,40.034868,6,0.001070,0.000858,...,19.698682,2.071563,0.075330,1.431262,5.423729,96.158192,0.225989,92.919021,3.013183,3.841808
144121,99999,51001,980200,Accomack VA,99999,-75.511891,37.838676,5,0.000892,1.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
144122,99999,56023,978200,Lincoln WY,99999,-110.665086,42.055432,6,0.001070,0.003009,...,16.852146,8.426073,0.000000,3.974563,18.918919,90.461049,1.112878,75.993641,13.354531,9.538951


## Aggregating by Zip Code

In [37]:
## Now that we've merged, our dataset contains multiple rows per zip, if there were multiple tracts per zip
## Let's aggregate on a zip to take the weighted average of the values we are evaluating


## From: https://www.statology.org/pandas-weighted-average/
def w_avg(df, values, weights):
    d = df[values]
    w = df[weights]
    return (d * w).sum() / w.sum()

In [38]:
full_zips = merged.groupby("Zip")
full_zips.count()

,county,tract,cntyname,zipname,intptlon,intptlat,pop10,afact,AFACT2,NAME,...,pct_internet_broadband_satellite,pct_internet_only_satellite,pct_internet_other,pct_internet_no_subscrp,pct_internet_none,pct_computer,pct_computer_with_dialup,pct_computer_with_broadband,pct_computer_no_internet,pct_no_computer
Zip,,,,,,,,,,,,,,,,,,,,,
01001,5,5,5,5,5,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
01002,9,9,9,9,9,9,9,9,9,9,...,8,8,8,8,8,8,8,8,8,8
01003,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
01005,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
01007,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99925,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
99926,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
99927,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [39]:
employment_rate = full_zips.apply(w_avg, 'employment_rate', "afact")

employment_rate

Zip
01001    62.054017
01002    59.048759
01003    33.100000
01005    66.100000
01007    70.799047
           ...    
99925    62.800000
99926    45.800000
99927    51.000000
99929    51.600000
99999    37.295530
Length: 32846, dtype: float64

In [40]:
## Confirm the values map to the output above
test = broadband_data.merge(employment_rate.rename('employment_rate'), on="Zip", how="left")
test[test.Zip=="01001"]

,Zip,Population,WiredCount_2020,Fwcount_2020,AllProviderCount_2020,Wired25_3_2020,Wired100_3_2020,All25_3_2020,All100_3,TestCount,...,All25_3_2015,All100_3.1,Total_Enrolled_Households,ST,COUNTY NAME,BROADBAND USAGE,ERROR RANGE (MAE)(+/-),ERROR RANGE (95%)(+/-),MSD,employment_rate
11398,01001,16769.0,3.0,1.0,10.0,1.0,1.0,4.0,2.0,279210.0,...,1.0,1.0,252.0,MA,Hampden,0.808,0.002,0.007,0.0,62.054017


In [41]:
## Let's test to confirm with 01001

merged[merged.Zip == "01001"][["NAME", "Zip", "tract","employment_rate","afact" ]]

,NAME,Zip,tract,employment_rate,afact
0,"Census Tract 8132.05, Hampden County, Massachu...",01001,813205,65.7,0.225118
1,"Census Tract 8132.06, Hampden County, Massachu...",01001,813206,64.0,0.017711
2,"Census Tract 8132.07, Hampden County, Massachu...",01001,813207,64.4,0.246467
3,"Census Tract 8132.08, Hampden County, Massachu...",01001,813208,69.4,0.174011
4,"Census Tract 8132.09, Hampden County, Massachu...",01001,813209,54.0,0.336693


In [42]:
broadband_data.columns

Index(['Zip', 'Population', 'WiredCount_2020', 'Fwcount_2020',
       'AllProviderCount_2020', 'Wired25_3_2020', 'Wired100_3_2020',
       'All25_3_2020', 'All100_3', 'TestCount', 'AverageMbps',
       'FastestAverageMbps', '%Access to Terrestrial Broadband',
       'Lowest Priced Terrestrial Broadband Plan', 'WiredCount_2015',
       'Fwcount_2015', 'AllProviderCount_2015', 'Wired25_3_2015',
       'Wired100_3_2015', 'All25_3_2015', 'All100_3.1',
       'Total_Enrolled_Households', 'ST', 'COUNTY NAME', 'BROADBAND USAGE',
       'ERROR RANGE (MAE)(+/-)', 'ERROR RANGE (95%)(+/-)', 'MSD'],
      dtype='object')

In [43]:
## Now what we want to do is add a new data column to the broadband dataset for (almost) each column
## in our census dataset using the weighted average calculation above

cols_to_keep = ['median_age_overall', 'median_age_male', 'median_age_female', 'employment_rate', 'median_income',
       'total_households', 'ave_household_size', 'ave_family_size',
       'total_population', 'median_house_value', 'pct_white',
       'pct_hisp_latino', 'pct_black', 'pct_native', 'pct_asian', 'pct_hi_pi',
       'pct_other_race', 'pct_two+_race', 'pct_rent_burdened', 'poverty_rate',
       'pct_pop_bachelors+', 'pct_pop_hs+', 'pct_internet',
       'pct_internet_dial_up', 'pct_internet_broadband_any_type',
       'pct_internet_cellular', 'pct_only_cellular',
       'pct_internet_broadband_fiber', 'pct_internet_broadband_satellite',
       'pct_internet_only_satellite', 'pct_internet_other',
       'pct_internet_no_subscrp', 'pct_internet_none', 'pct_computer',
       'pct_computer_with_dialup', 'pct_computer_with_broadband',
       'pct_computer_no_internet', 'pct_no_computer','pct_health_ins_children', 'pct_health_ins_19_64',
       'pct_health_ins_65+', 'total_pop2']

for col in cols_to_keep:
    temp = full_zips.apply(w_avg, col, "afact")
    broadband_data = broadband_data.merge(temp.rename(col), on="Zip", how="left")

In [44]:
broadband_data.columns

Index(['Zip', 'Population', 'WiredCount_2020', 'Fwcount_2020',
       'AllProviderCount_2020', 'Wired25_3_2020', 'Wired100_3_2020',
       'All25_3_2020', 'All100_3', 'TestCount', 'AverageMbps',
       'FastestAverageMbps', '%Access to Terrestrial Broadband',
       'Lowest Priced Terrestrial Broadband Plan', 'WiredCount_2015',
       'Fwcount_2015', 'AllProviderCount_2015', 'Wired25_3_2015',
       'Wired100_3_2015', 'All25_3_2015', 'All100_3.1',
       'Total_Enrolled_Households', 'ST', 'COUNTY NAME', 'BROADBAND USAGE',
       'ERROR RANGE (MAE)(+/-)', 'ERROR RANGE (95%)(+/-)', 'MSD',
       'median_age_overall', 'median_age_male', 'median_age_female',
       'employment_rate', 'median_income', 'total_households',
       'ave_household_size', 'ave_family_size', 'total_population',
       'median_house_value', 'pct_white', 'pct_hisp_latino', 'pct_black',
       'pct_native', 'pct_asian', 'pct_hi_pi', 'pct_other_race',
       'pct_two+_race', 'pct_rent_burdened', 'poverty_rate',
      

In [45]:
# Drop unnecessary columns
# Population was from the Broadband Now data but we've pulled this from the Census in total_population
cols_to_drop = ["Population"]
output = broadband_data.drop(columns=cols_to_drop)
output

,Zip,WiredCount_2020,Fwcount_2020,AllProviderCount_2020,Wired25_3_2020,Wired100_3_2020,All25_3_2020,All100_3,TestCount,AverageMbps,...,pct_internet_none,pct_computer,pct_computer_with_dialup,pct_computer_with_broadband,pct_computer_no_internet,pct_no_computer,pct_health_ins_children,pct_health_ins_19_64,pct_health_ins_65+,total_pop2
0,29639,3.0,0.0,8.0,3.0,3.0,5.0,3.0,163.0,93.12,...,25.562553,82.493936,0.074843,67.267894,15.151199,17.506064,88.950971,86.189438,100.000000,4306.115164
1,29620,6.0,0.0,11.0,5.0,3.0,7.0,3.0,2536.0,212.50,...,27.338094,80.712703,0.000575,65.819564,14.892564,19.287297,93.098385,81.591357,100.000000,5274.299089
2,29659,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,25.268432,85.254116,0.000000,61.775233,23.478883,14.745884,100.000000,83.900000,100.000000,3464.000000
3,29638,6.0,1.0,13.0,4.0,4.0,6.0,4.0,272.0,82.79,...,24.384813,81.319740,0.404128,70.247521,10.668091,18.680260,93.406966,84.480742,100.000000,3849.135001
4,29628,4.0,0.0,8.0,3.0,2.0,5.0,2.0,100.0,51.12,...,25.324715,78.286085,0.000000,67.895454,10.390630,21.713915,97.927401,78.157411,100.000000,2748.179543
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32648,78839,3.0,3.0,11.0,1.0,1.0,4.0,1.0,1392.0,48.94,...,36.704729,72.890747,0.701018,53.119484,19.070245,27.109253,95.447426,75.592370,99.127211,5622.838142
32649,78872,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,53.645195,74.825080,0.000000,38.861333,35.963747,25.174920,99.962206,59.619735,99.946457,1614.839963
32650,57622,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,43.899204,65.517241,0.000000,54.774536,10.742706,34.482759,88.700000,59.700000,97.300000,2791.000000
32651,57748,4.0,1.0,9.0,2.0,2.0,5.0,2.0,NaN,NaN,...,35.807258,72.147216,0.224401,62.497367,9.425449,27.852784,91.466664,67.164772,98.004716,3285.606370


In [46]:
## Testing it worked out

In [47]:
merged[merged.Zip=="02144"][["Zip", "pct_pop_hs+", "pct_internet", "median_age_overall", "afact"]]

,Zip,pct_pop_hs+,pct_internet,median_age_overall,afact
1347,02144,85.564304,86.371100,32.8,0.006027
1348,02144,91.255553,94.227642,31.1,0.233017
1349,02144,90.579710,88.034188,30.6,0.068185
1350,02144,87.792725,91.606960,22.1,0.115357
1351,02144,63.452602,78.315350,31.9,0.253150
1352,02144,89.901335,90.066964,31.4,0.076514
1353,02144,92.251309,92.433796,29.9,0.136076
1354,02144,91.893954,94.604811,29.8,0.106525
1355,02144,82.485747,87.842190,33.2,0.005148


In [48]:
output[output.Zip=="02144"][["Zip", "pct_pop_hs+", "pct_internet", "median_age_overall"]]

,Zip,pct_pop_hs+,pct_internet,median_age_overall
19252,02144,83.792124,88.87233,29.972451


## Merging with Rural/Urban Data

In [49]:
## merging of rural/urban data
rural = pd.read_excel("../data/zips_rural_urban.xlsx", sheet_name="Data", converters={'ZIP_CODE' : lambda x: str(x)})
rural = rural.rename(columns={"ZIP_CODE" : "Zip"})
rural

,Zip,STATE,ZIP_TYPE,RUCA1,RUCA2
0,00001,AK,Zip Code Area,10,10.0
1,00002,AK,Zip Code Area,10,10.0
2,00003,AK,Zip Code Area,10,10.0
3,00004,AK,Zip Code Area,10,10.0
4,00005,AK,Zip Code Area,10,10.0
...,...,...,...,...,...
41159,99926,AK,Zip Code Area,10,10.0
41160,99927,AK,Zip Code Area,10,10.0
41161,99928,AK,Post Office or large volume customer,4,4.0
41162,99929,AK,Zip Code Area,10,10.0


In [50]:
output = output.merge(rural, how="left", on="Zip")

In [51]:
output.to_csv("../data/weighted_merged_all.csv", index=False)